In [1]:
import pandas as pd
import numpy as np
import re
import requests

from concurrent.futures import ThreadPoolExecutor
import time

C:\Users\鲁鲁\AppData\Local\Temp\ipykernel_13128\421108197.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data=pd.read_excel('../DataScientists.xls')

In [3]:
data

,name,country,institution,dblp,expertise
0,aaron elmore,united states,university of chicago,https://dblp.org/pers/e/Elmore:Aaron_J=.html,NaN
1,abdalghani abujabal,germany,amazon alexa,https://dblp.org/pers/a/Abujabal:Abdalghani.html,NaN
2,abdul quamar,united states,ibm research almaden,https://dblp.org/pers/q/Quamar:Abdul.html,NaN
3,abdulhakim qahtan,netherlands,utrecht university,https://dblp.org/pid/121/4198.html,NaN
4,abhijnan chakraborty,germany,max planck institute for software systems,https://dblp.org/pers/c/Chakraborty:Abhijnan.html,NaN
...,...,...,...,...,...
1215,zixian huang,china,nanjing university,https://dblp.uni-trier.de/pers/hd/h/Huang:Zixian,NaN
1216,zoi kaoudi,qatar,qatar computing research institute,https://dblp.org/pers/k/Kaoudi:Zoi.html,NaN
1217,zoi kaoudi,germany,tu berlin,https://dblp.org/pers/k/Kaoudi:Zoi.html,NaN
1218,zsolt istvan,germany,tu darmstadt,https://dblp.org/pid/50/10703.html,NaN


In [4]:
url_list=[]
urls=data['dblp']
#print(len(urls))
#remove repeat in urls
urls=set(urls)
#len(urls)

In [5]:
def url_type(url):
    t=url.split('/')[3].split('/')[0]
    return t

In [8]:
counter429=0
disappear=set()
counter=0
def get_final_url(ori_url):
    global counter429,counter
    utype=url_type(ori_url)
    time.sleep(1)
    url=requests.get(ori_url,timeout=5)
    if url.status_code==429:
        time.sleep(int(url.headers['Retry-After'])+10)
        return get_final_url(ori_url)
    elif url.status_code==410 or url.status_code==404:
        print(url.status_code, url.url)
        disappear.add(ori_url)
        return None
    elif url.status_code!=200:
        print(url.status_code,url.url)
        return get_final_url(ori_url)
    elif url.status_code==200:
        counter429 =0
        counter+=1
        print(counter,url.url)
    else:
        pass
    return url.url

In [9]:
with ThreadPoolExecutor(max_workers=5) as ex:
    url_list=list(ex.map(get_final_url, urls))

1 https://dblp.org/pid/06/9379.html
2 https://dblp.org/pid/m/RJMiller.html
3 https://dblp.org/pid/07/1181.html
4 https://dblp.uni-trier.de/pid/r/MirekRiedewald.html
5 https://dblp.uni-trier.de/pid/75/5478.html
6 https://dblp.org/pid/18/2478.html
7 https://dblp.org/pid/80/2320.html
8 https://dblp.org/pid/m/SamuelMadden.html
9 https://dblp.uni-trier.de/pid/35/1895-1.html
10 https://dblp.org/pid/55/3364.html
11 https://dblp.org/pid/g/MinosNGarofalakis.html
12 https://dblp.uni-trier.de/pid/08/1790.html
13 https://dblp.org/pid/k/BettinaKemme.html
14 https://dblp.org/pid/23/3743.html
15 https://dblp.org/pid/14/4251.html
16 https://dblp.org/pid/127/6172-1.html
17 https://dblp.org/pid/01/4316-1.html
18 https://dblp.org/pid/128/2991.html
19 https://dblp.org/pid/37/3024.html
20 https://dblp.org/pid/97/9726.html
21 https://dblp.org/pid/75/887.html
22 https://dblp.org/pid/d/AmolDeshpande.html
23 https://dblp.uni-trier.de/pid/59/3814.html
24 https://dblp.org/pid/83/8652.html
25 https://dblp.uni-tri

In [10]:
print(len(url_list))
url_list=set(url_list)
print(len(url_list))
url_list=list(url_list)
urls1=pd.Series(url_list)


1079
1059


In [25]:
#remove None
urls1=urls1.dropna()
print(len(urls1))
urls1.to_csv('../Result/cleaned_url.csv',index=False)

1058


In [22]:
disappear=list(disappear)
disappear=pd.Series(disappear)
disappear=disappear.dropna()
disappear.to_csv('../Result/disappear.csv',index=False)

In [26]:
urls1

0       https://dblp.uni-trier.de/pid/66/3027.html
1                https://dblp.org/pid/06/9379.html
2                https://dblp.org/pid/07/1181.html
3             https://dblp.org/pid/m/RJMiller.html
4       https://dblp.uni-trier.de/pid/84/2154.html
                           ...                    
1054            https://dblp.org/pid/124/7268.html
1055            https://dblp.org/pid/139/1128.html
1056            https://dblp.org/pid/191/6085.html
1057    https://dblp.uni-trier.de/pid/32/6122.html
1058             https://dblp.org/pid/73/2015.html
Length: 1058, dtype: object

In [27]:
disappear

0                  https://dblp.org/pers/h/Hui:Kai.html
1        https://dblp.org/pers/w/Weidlich:Matthias.html
2     https://dblp.uni-trier.de/pers/p/Petrov:Ilia.html
3            https://dblp.org/pers/m/Mansour:Essam.html
4                     https://dblp.org/pid/98/5721.html
5     https://dblp.uni-trier.de/pers/b/Barbosa:Denil...
6            https://dblp.dagstuhl.de/pid/y/TingYu.html
7                    https://dblp.org/pid/148/7268.html
8                     https://dblp.org/pid/92/2769.html
9                     https://dblp.org/pid/39/1380.html
10           https://dblp.org/pers/j/Jung:Hyungsoo.html
11      https://dblp.uni-trier.de/pers/t/Tao:Yufei.html
12            https://dblp.org/pers/z/Zhang:Ruqing.html
13                   https://dblp.org/pid/284/0968.html
14                   https://dblp.org/pid/161/0102.html
15    https://dblp.uni-trier.de/pers/hd/n/Nikolic:Milos
16    https://dblp.uni-trier.de/pers/c/Chakraborty:A...
dtype: object